# Neural Style Transfer — Interactive (CPU)
Select a style from `data/style/`, enter a content image path, and run the transfer.
This notebook imports the functions from `nst_cpu_main.py` and saves results under `outputs/runs/`.

In [1]:
import os
# Enforce CPU-only and quieter TF logs before importing TensorFlow inside nst
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

from pathlib import Path
from IPython.display import display, Markdown
import nst_cpu_main as nst

# Base configuration (edit as needed)
cfg = nst.Config(
    image_size=256,
    steps=100,
    alpha=1.0,
    beta=2000.0,
    gamma=10.0,
    learning_rate=0.02,
    save_every=25,
    seed=42,
)

nst.ensure_dirs(cfg)
nst.set_global_seed(cfg.seed)
nst.print_config(cfg)


=== Neural Style Transfer (CPU) - Run Config ===
content_dir: data\content
style_dir: data\style
outputs_runs_dir: outputs\runs
outputs_plots_dir: outputs\plots
metrics_dir: metrics
image_size: 256
steps: 100
alpha: 1.0
beta: 2000.0
gamma: 10.0
seed: 42
resize_policy: square-resize
learning_rate: 0.02
log_every: 25
check_every: 50
early_stop_from_step: 300
early_stop_rel_improve: 0.005
save_every: 25
do_sanity_tests: True
do_batch_grid: True
do_portfolio_gallery: True
do_analysis_plots: True


In [3]:
# Interactive UI (ipywidgets)
try:
    import ipywidgets as widgets
    from IPython.display import display, Image as IPyImage, clear_output
    import traceback

    exts = {'.jpg', '.jpeg', '.png', '.bmp'}
    style_files = sorted([p for p in cfg.style_dir.glob('**/*') if p.suffix.lower() in exts])
    if not style_files:
        print(f'No style images found in {cfg.style_dir}. Add files and re-run this cell.')
    style_options = [(p.name, p) for p in style_files]

    content_default = ''
    content_sample = next((p for p in cfg.content_dir.glob('**/*') if p.suffix.lower() in exts), None)
    if content_sample:
        content_default = str(content_sample)

    style_dd = widgets.Dropdown(options=style_options, description='Style:')
    content_txt = widgets.Text(value=content_default, description='Content:')
    size_dd = widgets.Dropdown(options=[256, 384, 512], value=cfg.image_size, description='Size:')
    steps_it = widgets.IntText(value=cfg.steps, description='Steps:')
    alpha_ft = widgets.FloatText(value=cfg.alpha, description='Alpha:')
    beta_ft = widgets.FloatText(value=cfg.beta, description='Beta:')
    gamma_ft = widgets.FloatText(value=cfg.gamma, description='Gamma:')
    run_btn = widgets.Button(description='Run NST', button_style='primary')
    out = widgets.Output()

    def on_run_clicked(b):
        with out:
            out.clear_output()
            try:
                content_path = Path(content_txt.value).expanduser()
                style_path = style_dd.value
                if not content_path.exists():
                    print(f'Content path not found: {content_path}')
                    return

                cfg2 = nst.Config(
                    content_dir=cfg.content_dir,
                    style_dir=cfg.style_dir,
                    outputs_runs_dir=cfg.outputs_runs_dir,
                    outputs_plots_dir=cfg.outputs_plots_dir,
                    metrics_dir=cfg.metrics_dir,
                    image_size=int(size_dd.value),
                    steps=int(steps_it.value),
                    alpha=float(alpha_ft.value),
                    beta=float(beta_ft.value),
                    gamma=float(gamma_ft.value),
                    seed=cfg.seed,
                    resize_policy=cfg.resize_policy,
                    learning_rate=cfg.learning_rate,
                    log_every=cfg.log_every,
                    check_every=cfg.check_every,
                    early_stop_from_step=cfg.early_stop_from_step,
                    early_stop_rel_improve=cfg.early_stop_rel_improve,
                    save_every=cfg.save_every,
                )

                nst.set_global_seed(cfg2.seed)
                print(f'Running: content={content_path.name} style={style_path.name} size={cfg2.image_size}px steps={cfg2.steps} beta={cfg2.beta}')
                nst.run_single_nst(cfg2, content_path, style_path)

                # Show final image
                base = nst._deterministic_base(cfg2, content_path.stem, style_path.stem)
                final_path = cfg2.outputs_runs_dir / f'{base}__final.png'
                if final_path.exists():
                    display(IPyImage(filename=str(final_path)))
                    print(f'Saved final: {final_path}')
                else:
                    print('Final image not found; check logs above.')
            except Exception:
                traceback.print_exc()

    run_btn.on_click(on_run_clicked)
    ui = widgets.VBox([
        style_dd,
        content_txt,
        widgets.HBox([size_dd, steps_it]),
        widgets.HBox([alpha_ft, beta_ft, gamma_ft]),
        run_btn,
        out
    ])
    display(ui)
except ImportError:
    print('ipywidgets not installed. You can install via: pip install ipywidgets, then restart the kernel.')
    print('Fallback usage:')
    print('from nst_cpu_main import run_single_nst, Config')
    print('run_single_nst(cfg, Path("path/to/your/content.jpg"), Path("data/style/your_style.jpg"))')
